In [1]:
from rdflib import Graph, URIRef
import numpy as np
import glob 

import tqdm.notebook as tq

ONLY_ORGANIC = True
TRAINING = True

graph = Graph()
for filename in glob.glob('only_organic_reduced_kgs/reduced_*' if ONLY_ORGANIC else 'reduced_kgs/reduced_*'):
    graph.load(filename,format=filename.split('.')[-1])
    
entities = sorted(list(set(graph.subjects()) | set(graph.objects())))
relations = sorted(list(set(graph.predicates())))
len(entities), len(relations), len(graph)

(48463, 113, 200039)

In [2]:
import pandas as pd

effect_data = pd.read_csv('only_organic_effect_data_extra.csv' if ONLY_ORGANIC else 'effect_data_extra.csv')
ent = set(map(str,entities))
effect_data = effect_data[effect_data['species'].isin(ent)]
effect_data = effect_data[effect_data['chemical'].isin(ent)]
effect_data = effect_data[effect_data['smiles_clusters']>=0].reset_index(drop=True)

In [3]:

from time import sleep
from rdflib.namespace import Namespace
schema = Namespace('https://schema.org/')

import sys
from SPARQLWrapper import SPARQLWrapper, JSON
from pubchempy import Compound

endpoint_url = "https://query.wikidata.org/sparql"

query = """select ?cas ?pc where {
  ?c wdt:P231 ?tmp ;
     wdt:P662 ?pc .
  bind(replace(?tmp,'-','') as ?cas)
}"""


def get_results(endpoint_url, query):
    user_agent = "WDQS-example Python/%s.%s" % (sys.version_info[0], sys.version_info[1])
    # TODO adjust user agent; see https://w.wiki/CX6
    sparql = SPARQLWrapper(endpoint_url, agent=user_agent)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    return sparql.query().convert()


extra_graph = Graph()

try:
    extra_graph.load('only_organic_physical_properties.ttl' if ONLY_ORGANIC else 'physical_properties.ttl',format='ttl')

except FileNotFoundError:

    results = get_results(endpoint_url, query)
    for result in tq.tqdm(results["results"]["bindings"]):
        chem_id = 'https://cfpub.epa.gov/ecotox/cas/'+result['cas']['value']
        if not chem_id in ent:
            continue
        vioxx = Compound.from_cid(int(result['pc']['value']))
        mm = vioxx.molecular_weight
        logp = vioxx.xlogp

        if mm:

            if mm < 600:
                extra_graph.add((URIRef(chem_id),URIRef('http://example.org/compoundIsHeavy'),schema['False']))
            else:
                extra_graph.add((URIRef(chem_id),URIRef('http://example.org/compoundIsHeavy'),schema['True']))

        if logp:
            if logp < 4:
                extra_graph.add((URIRef(chem_id),URIRef('http://example.org/compoundLogPCategory'),URIRef('http://example.org/low')))
            elif logp < 5.5:
                extra_graph.add((URIRef(chem_id),URIRef('http://example.org/compoundLogPCategory'),URIRef('http://example.org/medium')))
            else:
                extra_graph.add((URIRef(chem_id),URIRef('http://example.org/compoundLogPCategory'),URIRef('http://example.org/high')))

    extra_graph.serialize('only_organic_physical_properties.ttl' if ONLY_ORGANIC else 'physical_properties.ttl',format='ttl')

graph += extra_graph
                

In [4]:
entities = sorted(list(set(graph.subjects()) | set(graph.objects())))
relations = sorted(list(set(graph.predicates())))
len(entities), len(relations), len(graph)

(48468, 115, 203338)

In [5]:
entity_mappings = {e:i for i,e in enumerate(entities)}
relation_mappings = {e:i for i,e in enumerate(relations)}
triples = np.asarray(list(map(lambda x: (entity_mappings[x[0]],
                                         relation_mappings[x[1]],
                                         entity_mappings[x[2]]),graph)))

In [6]:
def min_distance_loss(w,epsilon=1.0):
        
    r = tf.reduce_sum(w*w, 1)

    r = tf.reshape(r, [-1, 1])
    D = r - 2*tf.matmul(w, tf.transpose(w)) + tf.transpose(r)
    D = D + tf.linalg.diag(epsilon * tf.ones(D.shape[0]))
    return tf.reduce_sum(tf.where(D<epsilon,1.0,0.0))/tf.cast(w.shape[1],tf.float32)

In [7]:
import tensorflow as tf
from embedding_model import DistMult, ComplEx, TransE, ConvE

In [8]:
M = len(entities)

def create_negative(positive,n=2):
    negative = np.repeat(positive,n,axis=0)
    negative[:,0] = np.random.randint(0,M,size=len(positive)*n)
    negative[:,2] = np.random.randint(0,M,size=len(positive)*n)
    return negative

In [9]:
%timeit create_negative(triples,n=1)

3.42 ms ± 60.2 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [10]:
def train_generator(t,batch_size=32,n=2):
    
    t = t[np.random.randint(0,len(t),size=len(t))]
    
    num_batches = len(t)//batch_size
    
    for i in range(num_batches):
        
        positive = t[i*batch_size:i*batch_size+batch_size]
        negative = create_negative(positive,n)
        X = np.concatenate([positive,negative],axis=0)
        y = np.concatenate([np.ones(len(positive)),-1*np.ones(len(negative))],axis=0)
        
        yield (X,y),y
        

In [ ]:

total_models = 10
total_dim = 200

if TRAINING:

    embeddings = []

    for n in tq.tqdm(range(total_models)):

        embedding_model = ComplEx(entities,relations,dim=total_dim//total_models)

        best_loss = float('inf')
        losses = []
        patience=10

        for i in tq.tqdm(range(100),leave=False):

            negative = create_negative(triples,n=32)

            X = np.concatenate([triples,negative],axis=0)
            y = np.concatenate([np.ones(len(triples)),-1*np.ones(len(negative))],axis=0)

            hist = embedding_model.fit((X,y),y,
                             batch_size=8192,
                             shuffle=True,        
                             verbose=0)

            l = hist.history['loss'][-1]
            losses.append(l)
            if l < best_loss:
                best_loss = l
                c = 0
            else:
                c += 1

            if c > patience: break

        embeddings.append(embedding_model.get_layer('entity_embedding').get_weights()[0])
        
    W = np.concatenate(embeddings,axis=1)
    np.save('organic_only_W.npy' if ONLY_ORGANIC else 'W.npy',W)
        
else:
    W = np.load('organic_only_W.npy' if ONLY_ORGANIC else 'W.npy')


In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
if TRAINING:
    plt.plot(losses)

In [ ]:
fps = {}

import sys
from SPARQLWrapper import SPARQLWrapper, JSON
from pubchempy import Compound

endpoint_url = "https://query.wikidata.org/sparql"

query = """select ?cas ?pc where {
  ?c wdt:P231 ?tmp ;
     wdt:P662 ?pc .
  bind(replace(?tmp,'-','') as ?cas)
}"""


def get_results(endpoint_url, query):
    user_agent = "WDQS-example Python/%s.%s" % (sys.version_info[0], sys.version_info[1])
    # TODO adjust user agent; see https://w.wiki/CX6
    sparql = SPARQLWrapper(endpoint_url, agent=user_agent)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    return sparql.query().convert()

try: 
    fps = pd.read_pickle('fingerprints.pkl')

except FileNotFoundError:
    
    results = get_results(endpoint_url, query)
    for result in tq.tqdm(results["results"]["bindings"]):
        chem_id = 'https://cfpub.epa.gov/ecotox/cas/'+result['cas']['value']
        if chem_id in set(effect_data.chemical.values): 
            vioxx = Compound.from_cid(int(result['pc']['value']))
            fps[chem_id] = vioxx.fingerprint
    pd.to_pickle(fps,'fingerprints.pkl')


In [ ]:
def to_bin(he): 
    scale = 16 ## equals to hexadecimal
    num_of_bits = 900
    return bin(int(he, scale))[2:].zfill(num_of_bits)

effect_data['fp'] = [to_bin(fps[c]) if c in fps else to_bin('0') for c in effect_data['chemical'].values]

In [ ]:
effect_data = effect_data[effect_data['fp']!=to_bin('0')]
effect_data.shape

In [ ]:
effect_data.head()

In [ ]:
data = effect_data[['fp','species','chemical','conc (mol/L)']].values

In [ ]:
data = np.asarray([[fp,
                    entity_mappings[URIRef(s)],
                    entity_mappings[URIRef(c)],
                    conc] for fp,s,c,conc in data])

In [ ]:
def custom_loss(true,pred):
    return tf.reduce_mean(tf.where(true-pred>1,abs(true-pred),(true-pred)**2))
    

In [ ]:
from tensorflow.keras import backend as K

epsilon = 1e-6

@tf.function
def r2_keras(y_true, y_pred):
    SS_res =  tf.reduce_sum((y_true-y_pred)**2)
    SS_tot = tf.reduce_sum((y_true-tf.reduce_mean(y_true))**2)
    return ( 1 - SS_res/(SS_tot + epsilon))

from tensorflow.keras.layers import Concatenate, LayerNormalization, BatchNormalization, Activation, Input, Dense, Dropout, Embedding
from tensorflow.keras.models import Model, Sequential
def mlp(hidden=[128]):
    
    model = Sequential()
    for h in hidden:
        model.add(Dense(h))
        model.add(BatchNormalization())
        model.add(Activation(tf.keras.activations.swish))
        model.add(Dropout(0.2))
    model.add(Dense(1))
    
    model.compile(optimizer='adam',loss='mae',metrics=[r2_keras])
    return model
   
m = mlp()
m.build(input_shape=(1,100))
m.summary()

In [ ]:
def create_autoencoder(dim):
    encoder = tf.keras.models.Sequential([Dropout(0.2),Dense(200)])
    model = tf.keras.models.Sequential([encoder,Dropout(0.2),Dense(dim,activation='sigmoid')])
    model.compile(optimizer='adam',loss='binary_crossentropy')
    return model, encoder
    

In [ ]:
def chunks(lst, n):
    for i in range(0, len(lst), n):
        yield lst[i:i + n]
        
def kfold_group(X,y,groups,k=5):
    
    group_idx = [np.random.permutation(np.where(groups==g)[0]) for g in np.unique(groups)]
    
    folds = []
    
    for j,g in enumerate(group_idx):
        g_fold = []
        for l in np.array_split(g,k):
            g_fold.append(l)
        folds.append(g_fold)
        
    for i in range(k):
        test = np.concatenate([folds[j][i] for j in range(len(group_idx))])                          
        train = np.concatenate([np.concatenate([folds[j][m] for j in range(len(group_idx))]) for m in range(k) if m!=i]).ravel()
        
        yield train,test
        

In [ ]:

chemicals = set(data[:,1].astype(int))
species = set(data[:,2].astype(int))
chemical_mapping = {k:i for i,k in enumerate(chemicals)}
species_mapping = {k:i for i,k in enumerate(species)}

def data_generator(X,Y,use_embedding=False):

    if use_embedding:
        x = np.asarray(list(map(lambda x:np.concatenate([W[x[0]],W[x[1]]],axis=0),X)))
    else:
        v1 = np.zeros((len(X),len(chemicals)))
        v2 = np.zeros((len(X),len(species)))
        for j,(a,b) in enumerate(X):
            v1[j,chemical_mapping[a]] = 1.0
            v2[j,species_mapping[b]] = 1.0
        x = np.concatenate([v1,v2],axis=1)

    return x,Y
    

In [ ]:
from sklearn.model_selection import KFold, GroupKFold, GroupShuffleSplit, LeaveOneGroupOut, StratifiedKFold
from sklearn.metrics import r2_score
import random

y = data[:,3].astype('float32')
X = data[:,1:3].astype(int)

groups = effect_data['smiles_clusters'].values
folds = 5
EPOCHS = 1000

seed = 42
np.random.seed(seed)
random.seed(seed)
tf.random.set_seed(seed)
N = 10

scores = []
bs = len(y)

oof = np.zeros(y.shape)
oof_embedding = np.zeros(y.shape)

for _ in tq.tqdm(range(N)):

    oof_this_seed = np.zeros(y.shape)
    oof_this_seed_embedding = np.zeros(y.shape)
    
    for i,(train,test) in tq.tqdm(enumerate(kfold_group(X,y,groups=groups,k=folds)),
                                  total=folds,
                                  desc='Folds',
                                  leave=False):

        
        model = mlp(hidden=[128,64])
        
        model.fit(*data_generator(X[train],y[train]),
                  validation_data=data_generator(X[test],y[test]),
                  epochs=EPOCHS,verbose=0,
                  batch_size=bs,shuffle=True,
                 callbacks=[tf.keras.callbacks.EarlyStopping('val_r2_keras',mode='max',patience=10,restore_best_weights=True)])

        p = model.predict(data_generator(X[test],y[test])[0],batch_size=bs).ravel()
        oof[test] += p/N
        oof_this_seed[test] += p

        model = mlp(hidden=[128,64])
        
        model.fit(*data_generator(X[train],y[train],use_embedding=True),
                  validation_data=data_generator(X[test],y[test],use_embedding=True),
                  epochs=EPOCHS,verbose=0,
                  batch_size=bs,shuffle=True,
                 callbacks=[tf.keras.callbacks.EarlyStopping('val_r2_keras',mode='max',patience=10,restore_best_weights=True)])
        
        p = model.predict(data_generator(X[test],y[test],use_embedding=True)[0],batch_size=bs).ravel()
        oof_embedding[test] += p/N
        oof_this_seed_embedding[test] += p
        
    scores.append([r2_score(y,oof_this_seed),r2_score(y,oof_this_seed_embedding)])
    
    print('Out-of-fold R^2',scores[-1])

s,s_embedding = np.split(np.asarray(scores),2,axis=1)

'Out-of-fold R^2',np.mean(s),'+-',np.std(s),'Embedding:',np.mean(s_embedding),'+-',np.std(s_embedding)

In [ ]:
pd.DataFrame(data=dict(species=effect_data.species,chemical=effect_data.chemical,prediction=oof)).to_csv('only_organic_predictions.csv' if ONLY_ORGANIC else 'predictions.csv')
pd.DataFrame(data=dict(species=effect_data.species,chemical=effect_data.chemical,prediction=oof_embedding)).to_csv('only_organic_predictions_embedding.csv' if ONLY_ORGANIC else 'predictions_embedding.csv')